In [ ]:
"""
File: train_model.ipynb
Author: We Can't Code
GitHub: https://github.com/slyeet03/SmartSpray
"""

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random

In [ ]:
TRAIN_DIR="archive/train/" # download https://www.kaggle.com/datasets/ashishmotwani/tomato and add the path to the training images

# Initialize TensorBoard

In [ ]:
tsb=tf.keras.callbacks.TensorBoard(log_dir="logs")

# Create string labels

In [ ]:
classes = [i for i in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, i))]

# Create the training dataset

In [ ]:
train=[]
for i in os.listdir(TRAIN_DIR):
    current_path=os.path.join(TRAIN_DIR,i)
    current_class=classes.index(i)
    for j in os.listdir(current_path):
        try:
            img = cv2.imread(os.path.join(TRAIN_DIR, i, j), cv2.IMREAD_COLOR)  
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert from BGR (OpenCV default) to RGB
            img = cv2.resize(img, (210,210))

        except:
            continue
        train.append([img,current_class])
random.shuffle(train)
x=[]
y=[]
for i,j in train:
    x.append(i)
    y.append(j)
x = np.array(x)/255.0
y = np.array(y)
print(f"Total Train Images : {len(x)}")

# Create model

In [ ]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64,(3,3),activation=tf.nn.relu,input_shape=(210,210,3)), # input 70x70 grayscale image
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64,(3,3),activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64,(3,3),activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(256,activation=tf.nn.relu),
    tf.keras.layers.Dense(len(classes),activation=tf.nn.softmax)
])

# Get summary of the model

In [ ]:
model.summary()

# Compile the model

In [ ]:
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])

# Train the model

In [ ]:
history=model.fit(x,y,epochs=100,callbacks=[tsb],validation_split=0.1)

# Accuracy graph

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Train","Validation"],loc="best")

# Loss graph

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Train","Validation"],loc="best")

# Predict on images

In [ ]:
def prepare(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (210,210))
    return img.reshape(-1,210,210,3) / 255.0

In [ ]:
TEST_DIR = "test/"

for img_name in os.listdir(TEST_DIR):
    img_path = os.path.join(TEST_DIR, img_name)
    
    try:
        prediction = model.predict(prepare(img_path))
        predicted_class = classes[int(np.argmax(prediction))]
      
        print(f"Image: {img_name} --> Predicted Class: {predicted_class}")
        
        plt.imshow(plt.imread(img_path))
        plt.title(predicted_class)
        plt.axis("off")
        plt.show()
    except Exception as e:
        print(f"Could not process {img_name}: {e}")


# Save the model

In [ ]:
model.save("64x3-tomato_leaves.h5")

# Covert model to TFLite model for edge devices

In [ ]:
tf_lite_converter=tf.lite.TFLiteConverter.from_keras_model(model)
with open("64x3-tomato_leaves.tflite","wb") as f:
    f.write(tf_lite_converter.convert())